In [1]:
import os
import re
import pandas as pd
import numpy as np
import kagglehub
from tqdm import tqdm
import time
import requests
from SPARQLWrapper import SPARQLWrapper, JSON
from urllib.parse import urlparse

# Load Data (from Kaggle and Github)

In [2]:
path = kagglehub.dataset_download("odedgolden/movielens-1m-dataset")
print("Path to dataset files:", path)

Path to dataset files: C:\Users\Admin\.cache\kagglehub\datasets\odedgolden\movielens-1m-dataset\versions\1


In [3]:
movies_path = os.path.join(path, "movies.dat")
ratings_path = os.path.join(path, "ratings.dat")
users_path = os.path.join(path, "users.dat")

In [4]:
df_movies = pd.read_csv(
    movies_path,
    sep="::",
    engine="python",
    encoding="ISO-8859-1",
    names=["movie_id", "title", "genres"]
)

df_ratings = pd.read_csv(
    ratings_path,
    sep="::",
    engine="python",
    encoding="ISO-8859-1",
    names=["user_id", "movie_id", "rating", "timestamp"]
)

df_users = pd.read_csv(
    users_path,
    sep="::",
    engine="python",
    encoding="ISO-8859-1",
    names=["user_id", "gender", "age", "occupation", "zip_code"]
)

print(df_movies.head())
print(df_ratings.head())
print(df_users.head())

   movie_id                               title                        genres
0         1                    Toy Story (1995)   Animation|Children's|Comedy
1         2                      Jumanji (1995)  Adventure|Children's|Fantasy
2         3             Grumpier Old Men (1995)                Comedy|Romance
3         4            Waiting to Exhale (1995)                  Comedy|Drama
4         5  Father of the Bride Part II (1995)                        Comedy
   user_id  movie_id  rating  timestamp
0        1      1193       5  978300760
1        1       661       3  978302109
2        1       914       3  978301968
3        1      3408       4  978300275
4        1      2355       5  978824291
   user_id gender  age  occupation zip_code
0        1      F    1          10    48067
1        2      M   56          16    70072
2        3      M   25          15    55117
3        4      M   45           7    02460
4        5      M   25          20    55455


In [6]:
# Load the TSV mapping file directly from GitHub
url = "https://raw.githubusercontent.com/sisinflab/LODrecsys-datasets/master/Movielens1M/MappingMovielens2DBpedia-1.2.tsv"

df_mapping = pd.read_csv(url, sep="\t", header=None, names=["index", "title", "dbpedia_uri"])
df_mapping.head()

,index,title,dbpedia_uri
0,781,Stealing Beauty (1996),http://dbpedia.org/resource/Stealing_Beauty
1,1799,Suicide Kings (1997),http://dbpedia.org/resource/Suicide_Kings
2,521,Romeo Is Bleeding (1993),http://dbpedia.org/resource/Romeo_Is_Bleeding
3,3596,Screwed (2000),http://dbpedia.org/resource/Screwed_(2000_film)
4,3682,Magnum Force (1973),http://dbpedia.org/resource/Magnum_Force


# Remove cold-start users/items

In [7]:
# Step 1: Filter positive interactions
df_positive = df_ratings[df_ratings['rating'] >= 4]

# Step 2: Compute user/item interaction counts
user_counts = df_positive['user_id'].value_counts()
item_counts = df_positive['movie_id'].value_counts()

# Step 3: Compute thresholds for bottom 10%
user_thresh = user_counts.quantile(0.10)
item_thresh = item_counts.quantile(0.10)

# Step 4: Get users/items to keep
active_users = user_counts[user_counts > user_thresh].index
active_items = item_counts[item_counts > item_thresh].index

# Step 5: Filter the dataframe
df_filtered = df_positive[
    (df_positive['user_id'].isin(active_users)) &
    (df_positive['movie_id'].isin(active_items))
]

print(f"Filtered interactions: {len(df_filtered)} / {len(df_positive)}")
print(f"Remaining users: {df_filtered['user_id'].nunique()}")
print(f"Remaining items: {df_filtered['movie_id'].nunique()}")


Filtered interactions: 565817 / 575281
Remaining users: 5390
Remaining items: 3125


# Train-Test Split

In [ ]:
# 80/20 temporal split per user
df_filtered = df_filtered.sort_values(['user_id','timestamp'])
train_parts, test_parts = [], []

for uid, grp in df_filtered.groupby('user_id'):
    n_total  = len(grp)
    n_train  = max(1, int(n_total * 0.8))   # at least one in train
    train_parts.append(grp.iloc[:n_train])
    test_parts .append(grp.iloc[n_train:])

df_train = pd.concat(train_parts).reset_index(drop=True)
df_test  = pd.concat(test_parts) .reset_index(drop=True)

# Compute a “confidence” weight from the original rating
# (so that a 5-star = 1.0 confidence, 4-star = 0.5)
for df in (df_train, df_test):
    df['weight'] = (df['rating'] - 3) / 2

# # Save output
df_train[['user_id', 'movie_id', 'weight']].to_csv("../data/train.txt", sep='\t', index=False, header=False)
df_test[['user_id', 'movie_id', 'weight']].to_csv("../data/test.txt", sep='\t', index=False, header=False, )

# Query metadata from DBpedia

In [9]:
# Join movies with the mapping file
df_mapping.rename(columns={"index": "movie_id"}, inplace=True)
df_merged = pd.merge(df_movies, df_mapping[['movie_id', 'dbpedia_uri']], on='movie_id', how='left')

In [10]:
# SPARQL setup
sparql = SPARQLWrapper("https://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)
sparql.setTimeout(20)

results_list = []

# Query director, actor, country and language for all movies
for uri in tqdm(df_merged['dbpedia_uri'].dropna().unique(), desc="Querying DBpedia"):
    try:
        sparql.setQuery(f"""
            SELECT ?director ?actor ?country ?language WHERE {{
              OPTIONAL {{ <{uri}> dbo:director ?director . }}
              OPTIONAL {{ <{uri}> dbo:starring ?actor . }}
              OPTIONAL {{ <{uri}> dbp:country ?country . }}
              OPTIONAL {{ <{uri}> dbp:language ?language . }}
            }}
        """)

        response = sparql.query().convert()
        for result in response["results"]["bindings"]:
            results_list.append({
                'dbpedia_uri': uri,
                'director': result.get('director', {}).get('value'),
                'actor': result.get('actor', {}).get('value'),
                'country': result.get('country', {}).get('value'),
                'language': result.get('language', {}).get('value'),
            })

    except Exception as e:
        print(f"[ERROR] {uri} → {e}")
        continue

    time.sleep(0.25)

Querying DBpedia: 100%|██████████| 3300/3300 [46:33<00:00,  1.18it/s]


In [11]:
df_enriched = pd.DataFrame(results_list)

# Map dbpedia_uri back to movie_id
uri_to_mid = dict(zip(df_mapping['dbpedia_uri'], df_mapping['movie_id']))
df_enriched['movie_id'] = df_enriched['dbpedia_uri'].map(uri_to_mid)

# Drop any rows where movie_id couldn’t be mapped
df_enriched.dropna(subset=['movie_id'], inplace=True)

# Drop rows where all metadata fields are missing
df_enriched.replace("", np.nan, inplace=True)
df_enriched = df_enriched[~df_enriched[['director', 'actor', 'country', 'language']].isna().all(axis=1)]

# Normalize URIs and text values
def normalize_literal(value):
    if pd.isna(value) or str(value).strip().lower() in ["", "none"]:
        return None
    if str(value).startswith("http"):
        return urlparse(value).path.split("/")[-1]
    return value.strip().replace(" ", "_")

for col in ['director', 'actor', 'country', 'language']:
    df_enriched[col] = df_enriched[col].apply(normalize_literal)

# Remove parentheses from director and actor names
def strip_parentheses(value):
    if pd.isna(value):
        return None
    return re.sub(r"\(.*?\)", "", value).strip("_")

df_enriched['director'] = df_enriched['director'].apply(strip_parentheses)
df_enriched['actor'] = df_enriched['actor'].apply(strip_parentheses)

# EXPLODE multi-valued country and language
multivalue_cols = ['country', 'language']
for col in multivalue_cols:
    df_enriched[col] = df_enriched[col].astype(str).str.split(r"[,/\n]")
    df_enriched = df_enriched.explode(col)
    df_enriched[col] = df_enriched[col].str.strip().str.replace(r"^[*_\s]+", "", regex=True)
    df_enriched[col] = df_enriched[col].replace(['None', 'nan', 'NaN', ''], np.nan)

# Clean country column
country_aliases = {
    'United_States_of_America': 'United_States',
    'USA': 'United_States',
    'U.S': 'United_States',
    'UK': 'United_Kingdom',
    'British_Hong_Kong': 'Hong_Kong',
    'Cinema_of_Hong_Kong': 'Hong_Kong',
    "People's_Republic_of_China": 'China',
    'West_Germany': 'Germany',
    'Soviet_Union': 'Russia',
    'Weimar_Republic': 'Germany',
    'Georgia_(country)': 'Georgia',
    'Australia_': 'Australia',
    'Canada_': 'Canada',
    'United_Kingdom_': 'United_Kingdom',
    'Ireland_': 'Ireland',
    'United_Statesref|Fight_Club': 'United_States',
    'a_film_by_the_American_studio_20th_Century_Fox': 'United_States',
    'is_often_found_in_databases_and_related_summaries_to_have_the_countries_US_and_Germany': 'United_States',
    'the_latter_being_ascribable_to_the_role_of_international_funding.|group="nb"': 'United_States'
}

def normalize_country(value):
    if pd.isna(value):
        return None
    return country_aliases.get(value, value)

df_enriched['country'] = df_enriched['country'].apply(normalize_country)

# Clean language column
language_aliases = {
    'English_language': 'English',
    'English_Language': 'English',
    'English_(language)': 'English',
    'French_language': 'French',
    'German_language': 'German',
    'Spanish_language': 'Spanish',
    'Italian_language': 'Italian',
    'Portuguese_Language': 'Portuguese',
    'Russian_language': 'Russian',
    'Gujarati_language': 'Gujarati',
    'Bengali_language': 'Bengali',
    'Latin_language': 'Latin',
    'Welsh_language': 'Welsh',
    'Vietnamese_language': 'Vietnamese',
    'Scottish_Gaelic_language': 'Scottish_Gaelic',
    'Georgian_language': 'Georgian',
    'Friulian_language': 'Friulian',
    'Hindustani_language': 'Hindustani',
    'Juǀʼhoan_language': 'Juǀʼhoan',
    'Serbo-Croatian_language': 'Serbo-Croatian',
    'Gullah_language': 'Gullah',
    'Crow_language': 'Crow',
    'Salish-Spokane-Kalispel_language': 'Salish-Spokane-Kalispel',
    'Artistic_language': 'Artistic_Language',
    'Silent_film': 'Silent',
    'Silent_with_English_intertitles': 'Silent',
    'English_intertitles': 'English',
    'German_intertitles': 'German',
    'Silent_film,_English_intertitles': 'Silent',
    'Silent__intertitles': 'Silent',
    'South_African_English': 'English',
    'Mandarin_Chinese': 'Chinese',
    'Japanese_language': 'Japanese',
    'Portuguese_Language': 'Portuguese',
    'Juǀʼhoan_language': 'Juǀʼhoan',
    'Irish_Gaelic': 'Irish',
    'Gaelic': 'Scottish_Gaelic',
    'Scots': 'English',
    'Slovak_and_Russian': 'Czech',
    'Pennsylvania_Dutch': 'German',
    'Kyakhta_Russian–Chinese_Pidgin': 'Russian',
    'Asháninka': 'Spanish',

}

def normalize_language(value):
    if pd.isna(value):
        return None
    return language_aliases.get(value, value)

df_enriched['language'] = df_enriched['language'].apply(normalize_language)

<ipython-input-11-6ce92eda37a4>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_enriched[col] = df_enriched[col].apply(normalize_literal)


In [12]:
metadata_cols = ['director', 'actor', 'language', 'country']
df_enriched_agg = df_enriched.groupby('movie_id')[metadata_cols].agg(lambda x: '|'.join(set(x.dropna().astype(str)))).reset_index()

# Combine all metadata

In [13]:
df_full = df_movies.merge(df_enriched_agg, on='movie_id', how='left')

multi_cols = ['genres', 'actor', 'language', 'country']
for col in multi_cols:
    if col in df_full.columns:
        df_full[col] = df_full[col].fillna('').str.split('|')

df_full['year'] = df_full['title'].str.extract(r"\((\d{4})\)").astype(int)
df_full['title'] = df_full['title'].str.replace(r"\s*\(\d{4}\)", "", regex=True).str.strip()

df_full.head()

,movie_id,title,genres,director,actor,language,country,year
0,1,Toy Story,"[Animation, Children's, Comedy]",NaN,[],[],[],1995
1,2,Jumanji,"[Adventure, Children's, Fantasy]",Joe_Johnston,"[Jonathan_Hyde, Kirsten_Dunst, Robin_Williams,...",[English],[United_States],1995
2,3,Grumpier Old Men,"[Comedy, Romance]",Howard_Deutch,"[Ann-Margret, Sophia_Loren, Daryl_Hannah, Jack...",[English],[United_States],1995
3,4,Waiting to Exhale,"[Comedy, Drama]",Forest_Whitaker,"[Whitney_Houston, Angela_Bassett, Loretta_Devi...",[English],[United_States],1995
4,5,Father of the Bride Part II,[Comedy],Charles_Shyer,"[George_Newbern, Martin_Short, Steve_Martin, E...",[English],[United_States],1995


In [ ]:
df_full.to_csv("../data/movies_metadata.csv", index=False)

In [ ]:
df_users.to_csv("../data/users.csv", index=False)